<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_two_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Collaborative Task Solving with Multiple Agents and Human Users

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate an application involving multiple agents and human users to work together and accomplish a task. `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. We create multiple `UserProxyAgent` instances which can represent different human users.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
# %pip install pyautogen~=0.1.1

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

It first looks for an environment variable of a specified name ("OAI_CONFIG_LIST" in this example) which needs to be a valid json string. If that variable is not found, it then looks for a json file with the same name. It filters the configs by models (you can filter by other keys as well).

The json looks like the following:
```json
[
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key here>"
    },
    {
        "model": "gpt-4",
        "api_key": "<your Azure OpenAI API key here>",
        "api_base": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-07-01-preview"
    },
    {
        "model": "gpt-4-32k",
        "api_key": "<your Azure OpenAI API key here>",
        "api_base": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-07-01-preview"
    }
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.


In [6]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
print(config_list)

[{'model': 'gpt-4', 'api_key': '5d050ffec2b94f5eb43c54c80149561e', 'api_base': 'https://futu-002-caeast-001.openai.azure.com/', 'api_type': 'azure', 'api_version': '2023-07-01-preview'}, {'model': 'gpt-4-32k', 'api_key': '5d050ffec2b94f5eb43c54c80149561e', 'api_base': 'https://futu-002-caeast-001.openai.azure.com/', 'api_type': 'azure', 'api_version': '2023-07-01-preview'}]


In [8]:
trail_id = 46
default_llm_config = {
        "request_timeout": 600,
        "seed": trail_id,
        # Excluding azure openai endpoints from the config list.
        # Change to `exclude="openai"` to exclude openai endpoints, or remove the `exclude` argument to include both.
        "config_list": config_list,
        "model": "gpt-4",  # make sure the endpoint you use supports the model
        "temperature": 0
    }
                      

## Construct Agents

We define `ask_expert` function to start a conversation between two agents and return a summary of the result. We construct an assistant agent named "assistant_for_expert" and a user proxy agent named "expert". We specify `human_input_mode` as "ALWAYS" in the user proxy agent, which will always ask for feedback from the expert user.

In [9]:
def ask_expert(message):
    """
    将一个任务(message)询问专家(ask_expert)
    """
    # 专家的助理(专家自己不干活，都是他的助理干活，他只发号施令，叫专家的助理干活)
    assistant_for_expert = autogen.AssistantAgent(
        name="assistant_for_expert",
        max_consecutive_auto_reply=10,
        code_execution_config={"work_dir": "assistant_for_expert"},
        llm_config=default_llm_config
    )
    # 专家本家，不干活只发命令
    expert = autogen.UserProxyAgent(
        name="expert",
        human_input_mode="ALWAYS",
        max_consecutive_auto_reply=10,
        code_execution_config={"work_dir": "expert"},
    )

    # 专家接收到了任务，将这个任务发送给他的助理
    expert.initiate_chat(assistant_for_expert, message=message)
    # TODO: stop_reply_at_receive没懂
    expert.stop_reply_at_receive(assistant_for_expert)
    # expert.human_input_mode, expert.max_consecutive_auto_reply = "NEVER", 0
    # final message sent from the expert
    # 整个问题解决完之后(可能涉及来回多步对话)，专家发出最后一个命令: 让助理将以上解决方案总结一下，并以一种容易理解的方式解释答案
    expert.send("summarize the solution and explain the answer in an easy-to-understand way", assistant_for_expert)
    # return the last message the expert received
    # 助理完成总结后，发送最后一条消息给专家，这个消息作为整个ask_expert(message)的返回结果
    return expert.last_message()["content"]


We construct another assistant agent named "assistant_for_student" and a user proxy agent named "student". We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent. We set the `functions` in `AssistantAgent` and `function_map` in `UserProxyAgent` to use the created `ask_expert` function.

For simplicity, the `ask_expert` function is defined to run locally. For real applications, the function should run remotely to interact with an expert user.

In [12]:
'''
刚才有:
1. 专家
2. 专家的助理

现在有:
3. 学生
4. 学生的助理
    - system_message不再使用DEFAULT_SYSTEM_MESSAGE, 而是改成更简短的
'''
llm_config_with_func = default_llm_config.copy()
llm_config_with_func["functions"] = [
            {
                "name": "ask_expert",
                "description": "ask expert when you can't solve the problem satisfactorily.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask expert. Make sure the question include enough context, such as the code and the execution result. The expert does not know the conversation between you and the user, unless you share the conversation with the expert.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ]

# 学生的助理有一个工具: ask_expert
assistant_for_student = autogen.AssistantAgent(
    name="assistant_for_student",
    system_message="You are a helpful assistant. Reply TERMINATE when the task is done.",
    llm_config=llm_config_with_func
)

# 学生提供给助理一个工具: 问专家
student = autogen.UserProxyAgent(
    name="student",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "student"},
    function_map={"ask_expert": ask_expert},
)

## Perform a task

We invoke the `initiate_chat()` method of the student proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal in the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the student proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [13]:
# the assistant receives a message from the student, which contains the task description
student.initiate_chat(
    assistant_for_student,
    message="""Find $a + b + c$, given that $x+y \\neq -1$ and 
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
""",
)

student (to assistant_for_student):

Find $a + b + c$, given that $x+y \neq -1$ and 
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant_for_student (to student):

To find the values of $a$, $b$, and $c$, we can solve the system of equations. However, the system of equations you provided is not in the standard form. The standard form of a system of linear equations is $ax + by = c$. 

Let's rewrite the equations in the standard form:

1. $ax + by - x = 7 - c$
2. $a + bx - 2x = 6y - cy$
3. $ay + b - y = 4x - cx$

Now, we can solve this system of equations. However, it seems that there might be a mistake in the equations you provided. The system of equations is not consistent, and it seems impossible to solve for $a$, $b$, and $c$ using these equations. 

Could you please check the equations again?

----------------------------------------------

KeyboardInterrupt: Interrupted by user

When the assistant needs to consult the expert, it suggests a function call to `ask_expert`. When this happens, a line like the following will be displayed:

***** Suggested function Call: ask_expert *****
